### Import libraries 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor

### Import data 

In [2]:
customer_df = pd.read_csv('./we_fn_use_c_marketing_customer_value_analysis.csv')
display(customer_df.head())
customer_df.info()

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9134 entries, 0 to 9133
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Customer                       9134 non-null   object 
 1   State                          9134 non-null   object 
 2   Customer Lifetime Value        9134 non-null   float64
 3   Response                       9134 non-null   object 
 4   Coverage                       9134 non-null   object 
 5   Education                      9134 non-null   object 
 6   Effective To Date              9134 non-null   object 
 7   EmploymentStatus               9134 non-null   object 
 8   Gender                         9134 non-null   object 
 9   Income                         9134 non-null   int64  
 10  Location Code                  9134 non-null   object 
 11  Marital Status                 9134 non-null   object 
 12  Monthly Premium Auto           9134 non-null   i

In [3]:
### Here I am just doing some changes I did in the previous labs

columns_df = []
for col in customer_df.columns:
    col = col.lower().replace(" ", "_")
    columns_df.append(col)
    
customer_df.columns = columns_df
customer_df

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,LA72316,California,23405.987980,No,Basic,Bachelor,2/10/11,Employed,M,71941,...,89,0,2,Personal Auto,Personal L1,Offer2,Web,198.234764,Four-Door Car,Medsize
9130,PK87824,California,3096.511217,Yes,Extended,College,2/12/11,Employed,F,21604,...,28,0,1,Corporate Auto,Corporate L3,Offer1,Branch,379.200000,Four-Door Car,Medsize
9131,TD14365,California,8163.890428,No,Extended,Bachelor,2/6/11,Unemployed,M,0,...,37,3,2,Corporate Auto,Corporate L2,Offer1,Branch,790.784983,Four-Door Car,Medsize
9132,UP19263,California,7524.442436,No,Extended,College,2/3/11,Employed,M,21941,...,3,0,3,Personal Auto,Personal L2,Offer3,Branch,691.200000,Four-Door Car,Large


In [4]:
customer_df['effective_to_date']=pd.to_datetime(customer_df['effective_to_date'])
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9134 entries, 0 to 9133
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   customer                       9134 non-null   object        
 1   state                          9134 non-null   object        
 2   customer_lifetime_value        9134 non-null   float64       
 3   response                       9134 non-null   object        
 4   coverage                       9134 non-null   object        
 5   education                      9134 non-null   object        
 6   effective_to_date              9134 non-null   datetime64[ns]
 7   employmentstatus               9134 non-null   object        
 8   gender                         9134 non-null   object        
 9   income                         9134 non-null   int64         
 10  location_code                  9134 non-null   object        
 11  marital_status   

In [5]:
customer_df.isna().sum()

customer                         0
state                            0
customer_lifetime_value          0
response                         0
coverage                         0
education                        0
effective_to_date                0
employmentstatus                 0
gender                           0
income                           0
location_code                    0
marital_status                   0
monthly_premium_auto             0
months_since_last_claim          0
months_since_policy_inception    0
number_of_open_complaints        0
number_of_policies               0
policy_type                      0
policy                           0
renew_offer_type                 0
sales_channel                    0
total_claim_amount               0
vehicle_class                    0
vehicle_size                     0
dtype: int64

In [6]:
customer_df = customer_df.drop(["policy_type"], axis = 1)
customer_df.columns

Index(['customer', 'state', 'customer_lifetime_value', 'response', 'coverage',
       'education', 'effective_to_date', 'employmentstatus', 'gender',
       'income', 'location_code', 'marital_status', 'monthly_premium_auto',
       'months_since_last_claim', 'months_since_policy_inception',
       'number_of_open_complaints', 'number_of_policies', 'policy',
       'renew_offer_type', 'sales_channel', 'total_claim_amount',
       'vehicle_class', 'vehicle_size'],
      dtype='object')

### We will start with removing outliers, if you have not already done so. We have discussed different methods to remove outliers. Use the one you feel more comfortable with, define a function for that. Use the function to remove the outliers and apply it to the dataframe.

In [7]:
iqr = np.nanpercentile(customer_df['customer_lifetime_value'],75) - np.nanpercentile(customer_df['customer_lifetime_value'],25)
upper_limit = np.nanpercentile(customer_df['customer_lifetime_value'],75) + 1.5*iqr
customer_df = customer_df[customer_df['customer_lifetime_value'] < upper_limit]

In [8]:
iqr = np.nanpercentile(customer_df['total_claim_amount'],75) - np.nanpercentile(customer_df['total_claim_amount'],25)
upper_limit = np.nanpercentile(customer_df['total_claim_amount'],75) + 1.5*iqr 
customer_df = customer_df[customer_df['total_claim_amount'] < upper_limit]

In [9]:
customer_df.shape

(7962, 23)

### Import sklearn train_test_split and separate the data

In [10]:
### X,y split

y = customer_df["total_claim_amount"]
x = customer_df.drop(["total_claim_amount"], axis = 1)

In [11]:
### Train-test split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 85)
print(x_train.shape)
print(x_test.shape)

(6369, 22)
(1593, 22)


### Normalize the continuous variables. You can use any one method you want

In [12]:
num_train = x_train.select_dtypes("number")
num_test = x_test.select_dtypes("number")

In [13]:
transformer = MinMaxScaler().fit(num_train)
num_train_norm = transformer.transform(num_train)
num_test_norm = transformer.transform(num_test)
num_train_norm = pd.DataFrame(num_train_norm, columns=num_train.columns, index=num_train.index)
num_test_norm = pd.DataFrame(num_test_norm, columns=num_test.columns, index=num_test.index)
num_train_norm.head()

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
5897,0.334152,0.703554,0.102679,0.314286,0.323232,0.0,0.625
1922,0.056194,0.315473,0.040179,0.771429,0.464646,0.0,0.000
4851,0.034984,0.630566,0.000000,0.142857,0.797980,0.0,0.000
328,0.051870,0.390502,0.031250,0.428571,0.656566,0.0,0.000
5582,0.255438,0.202599,0.044643,0.000000,0.646465,0.0,0.250


### Encode the categorical variables (See the hint below for encoding categorical data!!!)

In [14]:
ord_train = x_train[["coverage", "employmentstatus", "location_code", "vehicle_size"]]
ord_test = x_test[["coverage", "employmentstatus", "location_code", "vehicle_size"]]
display(ord_train.head())

,coverage,employmentstatus,location_code,vehicle_size
5897,Extended,Employed,Urban,Medsize
1922,Basic,Employed,Rural,Medsize
4851,Basic,Employed,Suburban,Medsize
328,Basic,Employed,Urban,Medsize
5582,Basic,Employed,Urban,Medsize


In [15]:
ord_train["coverage"] = ord_train["coverage"].map({"Basic" : 0, "Extended" : 0.5, "Premium" : 1})
ord_train["employmentstatus"] = ord_train["employmentstatus"].map({"Employed" : 1, "Unemployed" : 0.75, "Medical Leave" : 0.5, "Retired" : 0.25, "Disabled" : 0})
ord_train["location_code"] = ord_train["location_code"].map({"Urban" : 1, "Suburban" : 0.5, "Rural" : 0})
ord_train["vehicle_size"] = ord_train["vehicle_size"].map({"Small" : 0, "Medsize" : 0.5, "Large" : 1})

for col in ord_train.columns:
    print(ord_train[col].value_counts())


0.0    4032
0.5    1849
1.0     488
Name: coverage, dtype: int64
1.00    4040
0.75    1523
0.50     301
0.00     294
0.25     211
Name: employmentstatus, dtype: int64
0.5    3947
0.0    1252
1.0    1170
Name: location_code, dtype: int64
0.5    4508
0.0    1210
1.0     651
Name: vehicle_size, dtype: int64


/var/folders/fw/gn4y0r6102q38_7vzn4l62mw0000gp/T/ipykernel_88777/3052367588.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ord_train["coverage"] = ord_train["coverage"].map({"Basic" : 0, "Extended" : 0.5, "Premium" : 1})
/var/folders/fw/gn4y0r6102q38_7vzn4l62mw0000gp/T/ipykernel_88777/3052367588.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ord_train["employmentstatus"] = ord_train["employmentstatus"].map({"Employed" : 1, "Unemployed" : 0.75, "Medical Leave" : 0.5, "Retired" : 0.25, "Disabled" : 

In [16]:
ord_test["coverage"] = ord_test["coverage"].map({"Basic" : 0, "Extended" : 0.5, "Premium" : 1})
ord_test["employmentstatus"] = ord_test["employmentstatus"].map({"Employed" : 1, "Unemployed" : 0.75, "Medical Leave" : 0.5, "Retired" : 0.25, "Disabled" : 0})
ord_test["location_code"] = ord_test["location_code"].map({"Urban" : 1, "Suburban" : 0.5, "Rural" : 0})
ord_test["vehicle_size"] = ord_test["vehicle_size"].map({"Small" : 0, "Medsize" : 0.5, "Large" : 1})


/var/folders/fw/gn4y0r6102q38_7vzn4l62mw0000gp/T/ipykernel_88777/610762940.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ord_test["coverage"] = ord_test["coverage"].map({"Basic" : 0, "Extended" : 0.5, "Premium" : 1})
/var/folders/fw/gn4y0r6102q38_7vzn4l62mw0000gp/T/ipykernel_88777/610762940.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ord_test["employmentstatus"] = ord_test["employmentstatus"].map({"Employed" : 1, "Unemployed" : 0.75, "Medical Leave" : 0.5, "Retired" : 0.25, "Disabled" : 0})
/v

In [17]:
cat_train = x_train.select_dtypes("object").drop(["customer", "coverage", "employmentstatus", "location_code", "vehicle_size"], axis = 1)
cat_test = x_test.select_dtypes("object").drop(["customer", "coverage", "employmentstatus", "location_code", "vehicle_size"], axis = 1)
cat_train.head()


,state,response,education,gender,marital_status,policy,renew_offer_type,sales_channel,vehicle_class
5897,Arizona,No,High School or Below,F,Married,Corporate L1,Offer4,Branch,Four-Door Car
1922,California,No,Bachelor,F,Single,Personal L2,Offer4,Branch,Four-Door Car
4851,California,No,College,M,Married,Personal L3,Offer2,Agent,Four-Door Car
328,Oregon,No,College,F,Divorced,Personal L2,Offer2,Agent,Four-Door Car
5582,Arizona,No,Master,M,Married,Personal L3,Offer2,Call Center,Four-Door Car


In [18]:
encoder = OneHotEncoder(handle_unknown='error', drop='first')
encoder.fit(cat_train)

cat_train_encoded = encoder.transform(cat_train).toarray()
cat_test_encoded  = encoder.transform(cat_test).toarray()

cat_train_encoded = pd.DataFrame(cat_train_encoded, columns = encoder.get_feature_names_out(), index=cat_train.index)
cat_test_encoded  = pd.DataFrame(cat_test_encoded, columns = encoder.get_feature_names_out(), index=cat_test.index)
display(cat_train_encoded.head())
display(cat_test_encoded.head())

,state_California,state_Nevada,state_Oregon,state_Washington,response_Yes,education_College,education_Doctor,education_High School or Below,education_Master,gender_M,...,renew_offer_type_Offer3,renew_offer_type_Offer4,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_Luxury Car,vehicle_class_Luxury SUV,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car
5897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1922,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4851,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
328,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


,state_California,state_Nevada,state_Oregon,state_Washington,response_Yes,education_College,education_Doctor,education_High School or Below,education_Master,gender_M,...,renew_offer_type_Offer3,renew_offer_type_Offer4,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_Luxury Car,vehicle_class_Luxury SUV,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car
8854,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
7045,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1600,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1135,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3270,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### The time variable can be useful. Try to transform its data into a useful one. Hint: Day week and month as integers might be useful.

In [19]:
date_train = x_train[["effective_to_date"]]
date_train['day'] = date_train['effective_to_date'].dt.strftime('%d')
date_train['day'] = pd.to_numeric(date_train['day'])
date_train['month'] = date_train['effective_to_date'].dt.strftime('%m')
date_train['month'] = pd.to_numeric(date_train['month'])
date_train['year'] = date_train['effective_to_date'].dt.strftime('%y')
date_train['year'] = pd.to_numeric(date_train['year'])
date_train = date_train[["day", "month", "year"]]
date_train

/var/folders/fw/gn4y0r6102q38_7vzn4l62mw0000gp/T/ipykernel_88777/1680248927.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_train['day'] = date_train['effective_to_date'].dt.strftime('%d')
/var/folders/fw/gn4y0r6102q38_7vzn4l62mw0000gp/T/ipykernel_88777/1680248927.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_train['day'] = pd.to_numeric(date_train['day'])
/var/folders/fw/gn4y0r6102q38_7vzn4l62mw0000gp/T/ipykernel_88777/1680248927.py:7: SettingWithCopyWarning: 
A value is trying to be se

,day,month,year
5897,26,2,11
1922,12,1,11
4851,19,2,11
328,20,1,11
5582,19,2,11
...,...,...,...
8074,19,1,11
1814,18,2,11
6692,10,2,11
6060,10,2,11


In [20]:
date_test = x_test[["effective_to_date"]]
date_test['day'] = date_test['effective_to_date'].dt.strftime('%d')
date_test['day'] = pd.to_numeric(date_test['day'])
date_test['month'] = date_test['effective_to_date'].dt.strftime('%m')
date_test['month'] = pd.to_numeric(date_test['month'])
date_test['year'] = date_test['effective_to_date'].dt.strftime('%y')
date_test['year'] = pd.to_numeric(date_test['year'])
date_test = date_test[["day", "month", "year"]]
date_test

/var/folders/fw/gn4y0r6102q38_7vzn4l62mw0000gp/T/ipykernel_88777/3522083186.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_test['day'] = date_test['effective_to_date'].dt.strftime('%d')
/var/folders/fw/gn4y0r6102q38_7vzn4l62mw0000gp/T/ipykernel_88777/3522083186.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_test['day'] = pd.to_numeric(date_test['day'])
/var/folders/fw/gn4y0r6102q38_7vzn4l62mw0000gp/T/ipykernel_88777/3522083186.py:4: SettingWithCopyWarning: 
A value is trying to be set on

,day,month,year
8854,21,2,11
7045,12,1,11
1600,22,1,11
1135,16,2,11
3270,22,2,11
...,...,...,...
1172,22,2,11
3024,26,1,11
255,18,2,11
842,24,2,11


### Since the model will only accept numerical data, check and make sure that every column is numerical, if some are not, change it using encoding.

In [21]:
print(num_train_norm.shape)
print(ord_train.shape)
print(cat_train_encoded.shape)
print(date_train.shape)

print(num_test_norm.shape)
print(ord_test.shape)
print(cat_test_encoded.shape)
print(date_test.shape)

(6369, 7)
(6369, 4)
(6369, 31)
(6369, 3)
(1593, 7)
(1593, 4)
(1593, 31)
(1593, 3)


In [22]:
print(num_train_norm.info())
print(ord_train.info())
print(cat_train_encoded.info())
print(date_train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6369 entries, 5897 to 5086
Data columns (total 7 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   customer_lifetime_value        6369 non-null   float64
 1   income                         6369 non-null   float64
 2   monthly_premium_auto           6369 non-null   float64
 3   months_since_last_claim        6369 non-null   float64
 4   months_since_policy_inception  6369 non-null   float64
 5   number_of_open_complaints      6369 non-null   float64
 6   number_of_policies             6369 non-null   float64
dtypes: float64(7)
memory usage: 398.1 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6369 entries, 5897 to 5086
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   coverage          6369 non-null   float64
 1   employmentstatus  6369 non-null   float64
 2   lo

In [23]:
print(num_test_norm.info())
print(ord_test.info())
print(cat_test_encoded.info())
print(date_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1593 entries, 8854 to 5801
Data columns (total 7 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   customer_lifetime_value        1593 non-null   float64
 1   income                         1593 non-null   float64
 2   monthly_premium_auto           1593 non-null   float64
 3   months_since_last_claim        1593 non-null   float64
 4   months_since_policy_inception  1593 non-null   float64
 5   number_of_open_complaints      1593 non-null   float64
 6   number_of_policies             1593 non-null   float64
dtypes: float64(7)
memory usage: 99.6 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1593 entries, 8854 to 5801
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   coverage          1593 non-null   float64
 1   employmentstatus  1593 non-null   float64
 2   loc

In [24]:
display(num_train_norm.head())
display(ord_train.head())
display(cat_train_encoded.head())
display(date_train.head())

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
5897,0.334152,0.703554,0.102679,0.314286,0.323232,0.0,0.625
1922,0.056194,0.315473,0.040179,0.771429,0.464646,0.0,0.000
4851,0.034984,0.630566,0.000000,0.142857,0.797980,0.0,0.000
328,0.051870,0.390502,0.031250,0.428571,0.656566,0.0,0.000
5582,0.255438,0.202599,0.044643,0.000000,0.646465,0.0,0.250


,coverage,employmentstatus,location_code,vehicle_size
5897,0.5,1.0,1.0,0.5
1922,0.0,1.0,0.0,0.5
4851,0.0,1.0,0.5,0.5
328,0.0,1.0,1.0,0.5
5582,0.0,1.0,1.0,0.5


,state_California,state_Nevada,state_Oregon,state_Washington,response_Yes,education_College,education_Doctor,education_High School or Below,education_Master,gender_M,...,renew_offer_type_Offer3,renew_offer_type_Offer4,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_Luxury Car,vehicle_class_Luxury SUV,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car
5897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1922,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4851,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
328,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


,day,month,year
5897,26,2,11
1922,12,1,11
4851,19,2,11
328,20,1,11
5582,19,2,11


In [25]:
display(num_test_norm.head())
display(ord_test.head())
display(cat_test_encoded.head())
display(date_test.head())

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
8854,0.795978,0.267184,0.272321,0.285714,0.242424,0.0,0.125
7045,0.384758,0.525335,0.017857,0.000000,0.767677,0.6,0.125
1600,0.392724,0.598703,0.584821,0.800000,0.626263,0.0,0.000
1135,0.063307,0.412441,0.044643,0.542857,0.131313,0.0,0.000
3270,0.176165,0.000000,0.031250,0.142857,0.727273,0.2,0.250


,coverage,employmentstatus,location_code,vehicle_size
8854,0.5,0.25,0.5,0.5
7045,0.0,1.00,0.0,0.5
1600,0.0,1.00,0.5,0.5
1135,0.0,1.00,0.5,0.0
3270,0.0,0.75,0.5,1.0


,state_California,state_Nevada,state_Oregon,state_Washington,response_Yes,education_College,education_Doctor,education_High School or Below,education_Master,gender_M,...,renew_offer_type_Offer3,renew_offer_type_Offer4,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_Luxury Car,vehicle_class_Luxury SUV,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car
8854,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
7045,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1600,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1135,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3270,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,day,month,year
8854,21,2,11
7045,12,1,11
1600,22,1,11
1135,16,2,11
3270,22,2,11


In [26]:
### Here I am concatenating all the tables

x_train = pd.concat([num_train_norm, ord_train, cat_train_encoded, date_train], axis = 1)
x_test = pd.concat([num_test_norm, ord_test, cat_test_encoded, date_test], axis = 1)
display(x_train.head())
display(x_test.head())
print(x_train.shape)
print(x_test.shape)

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,coverage,employmentstatus,location_code,...,sales_channel_Call Center,sales_channel_Web,vehicle_class_Luxury Car,vehicle_class_Luxury SUV,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car,day,month,year
5897,0.334152,0.703554,0.102679,0.314286,0.323232,0.0,0.625,0.5,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26,2,11
1922,0.056194,0.315473,0.040179,0.771429,0.464646,0.0,0.000,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,1,11
4851,0.034984,0.630566,0.000000,0.142857,0.797980,0.0,0.000,0.0,1.0,0.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19,2,11
328,0.051870,0.390502,0.031250,0.428571,0.656566,0.0,0.000,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,1,11
5582,0.255438,0.202599,0.044643,0.000000,0.646465,0.0,0.250,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,19,2,11


,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,coverage,employmentstatus,location_code,...,sales_channel_Call Center,sales_channel_Web,vehicle_class_Luxury Car,vehicle_class_Luxury SUV,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car,day,month,year
8854,0.795978,0.267184,0.272321,0.285714,0.242424,0.0,0.125,0.5,0.25,0.5,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,21,2,11
7045,0.384758,0.525335,0.017857,0.000000,0.767677,0.6,0.125,0.0,1.00,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,12,1,11
1600,0.392724,0.598703,0.584821,0.800000,0.626263,0.0,0.000,0.0,1.00,0.5,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,22,1,11
1135,0.063307,0.412441,0.044643,0.542857,0.131313,0.0,0.000,0.0,1.00,0.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16,2,11
3270,0.176165,0.000000,0.031250,0.142857,0.727273,0.2,0.250,0.0,0.75,0.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22,2,11


(6369, 45)
(1593, 45)


In [27]:
x_train.isna().sum()

customer_lifetime_value           0
income                            0
monthly_premium_auto              0
months_since_last_claim           0
months_since_policy_inception     0
number_of_open_complaints         0
number_of_policies                0
coverage                          0
employmentstatus                  0
location_code                     0
vehicle_size                      0
state_California                  0
state_Nevada                      0
state_Oregon                      0
state_Washington                  0
response_Yes                      0
education_College                 0
education_Doctor                  0
education_High School or Below    0
education_Master                  0
gender_M                          0
marital_status_Married            0
marital_status_Single             0
policy_Corporate L2               0
policy_Corporate L3               0
policy_Personal L1                0
policy_Personal L2                0
policy_Personal L3          

### Try a simple linear regression with all the data to see whether we are getting good results.

In [28]:
lm = LinearRegression()
lm.fit(x_train, y_train)
y_pred_test = lm.predict(x_test)
r2_test = r2_score(y_test, y_pred_test)
print(r2_test)

0.5336189054098815


### Great! Now define a function that takes a list of models and train (and tests) them so we can try a lot of them without repeating code.

### Use the function to check LinearRegressor and KNeighborsRegressor.

### You can check also the MLPRegressor for this task!

In [69]:
def different_models(lst):
    models = {'k': [],'distance': [], 'p': [], 'model': [], "score": []}
    for mod in lst:
        if mod == lst[0] or mod == lst[2]:
            model = mod
            model.fit(x_train, y_train)
            y_pred_test = model.predict(x_test)
            r2_test = r2_score(y_test, y_pred_test)
            models['k'].append("none")
            models['distance'].append("none")
            models['p'].append("none")
            models['model'].append(model)
            models['score'].append(r2_test)
        elif mod == lst[1]:
            for k in range(2,5):
                for distance in ['uniform','distance']: 
                    for p in range(1,4):
                        model = KNeighborsRegressor(n_neighbors=k, weights = distance, p = p)
                        model.fit(x_train, y_train)
                        y_pred_test = model.predict(x_test)
                        r2_test = r2_score(y_test, y_pred_test)
                        models['k'].append(k)
                        models['distance'].append(distance)
                        models['p'].append(p)
                        models['model'].append(model)
                        models['score'].append(r2_test)
    results = pd.DataFrame(models)
    return results.sort_values(by = ["score"], ascending=False)


In [79]:
mods = [LinearRegression(), KNeighborsRegressor(), MLPRegressor()]
different_models(mods)

/Users/origenolet/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


,k,distance,p,model,score
19,none,none,none,MLPRegressor(),0.607868
0,none,none,none,LinearRegression(),0.533619
16,4,distance,1,"KNeighborsRegressor(n_neighbors=4, p=1, weight...",0.365997
13,4,uniform,1,"KNeighborsRegressor(n_neighbors=4, p=1)",0.360145
10,3,distance,1,"KNeighborsRegressor(n_neighbors=3, p=1, weight...",0.317603
7,3,uniform,1,"KNeighborsRegressor(n_neighbors=3, p=1)",0.314697
1,2,uniform,1,"KNeighborsRegressor(n_neighbors=2, p=1)",0.224354
4,2,distance,1,"KNeighborsRegressor(n_neighbors=2, p=1, weight...",0.222751
17,4,distance,2,"KNeighborsRegressor(n_neighbors=4, weights='di...",0.207366
14,4,uniform,2,KNeighborsRegressor(n_neighbors=4),0.203402


In [67]:
### It seems that the MLPRegressor with default values performs better than any other 
### model. It is followed by the linear regression model, and then the KNeighborsRegressor
### performs actually quite badly. From all the KNeighborsRegressor variations, it seems
### that the model performs the best with k = 4, weighing by distance, and with the manhattan 
### distance. 

In [84]:
def different_models(lst):
    models = {'k': [],'distance': [], 'p': [], 'model': [], "score": []}
    for mod in lst:
        if mod == "LinearRegression()":
            model = LinearRegression()
            model.fit(x_train, y_train)
            y_pred_test = model.predict(x_test)
            r2_test = r2_score(y_test, y_pred_test)
            models['k'].append("none")
            models['distance'].append("none")
            models['p'].append("none")
            models['model'].append(model)
            models['score'].append(r2_test)
        elif mod == "KNeighborsRegressor()":
            for k in range(2,5):
                for distance in ['uniform','distance']: 
                    for p in range(1,4):
                        model = KNeighborsRegressor(n_neighbors=k, weights = distance, p = p)
                        model.fit(x_train, y_train)
                        y_pred_test = model.predict(x_test)
                        r2_test = r2_score(y_test, y_pred_test)
                        models['k'].append(k)
                        models['distance'].append(distance)
                        models['p'].append(p)
                        models['model'].append(model)
                        models['score'].append(r2_test)
        elif mod == "MLPRegressor()":
            model = MLPRegressor()
            model.fit(x_train, y_train)
            y_pred_test = model.predict(x_test)
            r2_test = r2_score(y_test, y_pred_test)
            models['k'].append("none")
            models['distance'].append("none")
            models['p'].append("none")
            models['model'].append(model)
            models['score'].append(r2_test)
    results = pd.DataFrame(models)
    return results.sort_values(by = ["score"], ascending=False)

In [85]:
mods = ["LinearRegression()", "KNeighborsRegressor()", "MLPRegressor()"]
different_models(mods)

/Users/origenolet/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


,k,distance,p,model,score
19,none,none,none,MLPRegressor(),0.592239
0,none,none,none,LinearRegression(),0.533619
16,4,distance,1,"KNeighborsRegressor(n_neighbors=4, p=1, weight...",0.365997
13,4,uniform,1,"KNeighborsRegressor(n_neighbors=4, p=1)",0.360145
10,3,distance,1,"KNeighborsRegressor(n_neighbors=3, p=1, weight...",0.317603
7,3,uniform,1,"KNeighborsRegressor(n_neighbors=3, p=1)",0.314697
1,2,uniform,1,"KNeighborsRegressor(n_neighbors=2, p=1)",0.224354
4,2,distance,1,"KNeighborsRegressor(n_neighbors=2, p=1, weight...",0.222751
17,4,distance,2,"KNeighborsRegressor(n_neighbors=4, weights='di...",0.207366
14,4,uniform,2,KNeighborsRegressor(n_neighbors=4),0.203402
